<a href="https://colab.research.google.com/github/IzaakGagnon/Integrated_Information_Testing/blob/main/Heuristics_For_Phi_With_Statistical_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install -U git+https://github.com/wmayner/pyphi.git@feature/iit-4.0
import numpy as np
import pyphi
import random
import time
pyphi.config.PROGRESS_BARS = False
pyphi.config.SHORTCIRCUIT_SIA = False
pyphi.config.WELCOME_OFF = True
pyphi.config.REPERTOIRE_DISTANCE = "GENERALIZED_INTRINSIC_DIFFERENCE"
pyphi.config.PARALLEL = False

  Cloning https://github.com/wmayner/pyphi.git (to revision feature/iit-4.0) to /tmp/pip-req-build-35ndrd2o
  Running command git clone --filter=blob:none --quiet https://github.com/wmayner/pyphi.git /tmp/pip-req-build-35ndrd2o
  Resolved https://github.com/wmayner/pyphi.git to commit 22498f368381782d24c0e46f0717bd78900b24a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyphi: filename=pyphi-2.0.0a1-py3-none-any.whl size=707860 sha256=318124909e1bcfbecda43b631dd8cb364e4e68d80f89c7688753c44d38faf4b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-m7sd0f9s/wheels/53/0c/22/b822eefec0e167f711096ad1eeecd6964cb6030a41bda0e07e
  Created wheel for Graphillion: filename=Graphillion-1.8-cp310-cp310-linux_x86_64.whl size=6957512 sha256=d8f4e87cc05da4d5967623ad42cefea1196

In [ ]:
### KDE Work:


from abc import get_cache_token
from pyphi.new_big_phi import *
import matplotlib.pyplot as plt
import functools
import itertools
from itertools import chain, product
import numpy as np
from more_itertools import distinct_permutations
from toolz import unique
from pyphi import combinatorics
from pyphi.cache import cache
from pyphi.conf import config, fallback
from pyphi.direction import Direction
from pyphi.models.cuts import (
    Bipartition,
    CompleteGeneralKCut,
    CompleteGeneralSetPartition,
    Cut,
    GeneralKCut,
    GeneralSetPartition,
    KPartition,
    Part,
    SystemPartition,
    Tripartition,)
from pyphi.partition import system_partition_types
from pyphi.registry import Registry
from itertools import combinations

### Full Implementation Of The Faster Bi-Partition Algorithm
@system_partition_types.register("BI-PARTITIONS")
def generate_all_bipartitions(node_indices, node_labels=None):


  ### Perform the trivial cut
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)

  _node_indices = set(range(len(node_indices)))


  ### Loop through the possible partitions of the network.
  for partition in get_bipartitions(_node_indices):
      for directions in product(Direction.all(), repeat=len(partition)):
            if Direction.CAUSE in directions and Direction.EFFECT in directions:

              ### Fill out cut Matrix for a partition-direction grouping
              cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
              for part, direction in zip(partition, directions):

                  nonpart = list(_node_indices - set(part))



                  if direction == Direction.CAUSE:
                      source, target = nonpart, part
                  else:
                      source, target = part, nonpart
                  cut_matrix[np.ix_(source, target)] = 1
                  if direction == Direction.BIDIRECTIONAL:
                      cut_matrix[np.ix_(target, source)] = 1



                    ### Yield all the cuts
              yield GeneralSetPartition(
                  node_indices,
                  cut_matrix,
                  node_labels=node_labels,
                  set_partition=partition,
              )



              ### Yield the cut ignored by the if (both directions in set)
  yield GeneralSetPartition(
                node_indices,
                np.ones((len(_node_indices), len(_node_indices))) - np.identity(len(_node_indices)),
                node_labels=node_labels,
                set_partition=partition,
            )


def generate_restricted_bipartitions(node_indices, node_labels=None):
    directions = [True, False]
    """Generate all unidirectional set partitions of a set of nodes."""

    # Trivial Cut
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)


    _node_indices = set(range(len(node_indices)))
    for partition in get_bipartitions(_node_indices):
        for direction_seq in product(directions, repeat=len(partition)):

            # Fill out cut matrix
            cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
            for part, direction in zip(partition, direction_seq):
                nonpart = list(_node_indices - set(part))
                if direction:
                    source, target = nonpart, part
                else:
                    source, target = part, nonpart
                cut_matrix[np.ix_(source, target)] = 1


                # yield partition
            yield GeneralSetPartition(
                node_indices,
                cut_matrix,
                node_labels=node_labels,
                set_partition=partition,
            )


# Only do unique ones- ignores the very last one
@system_partition_types.register("UNIDIRECTIONAL_BIPARTITIONS")
@functools.wraps(generate_restricted_bipartitions)
def unidirectional_set_bipartitions(node_indices, node_labels=None):
    # TODO(4.0) generate properly without using set
    yield from unique(
        generate_restricted_bipartitions(node_indices, node_labels=node_labels)
    )


def get_phi_distribution(network, state):
  subsystem = pyphi.Subsystem(network,state)
  distribution = []

  def _has_no_cause_or_effect(system_state):
    reasons = []
    for direction, reason in zip(
        Direction.both(),
        [ShortCircuitConditions.NO_CAUSE, ShortCircuitConditions.NO_EFFECT],
    ):
        if system_state[direction].intrinsic_information <= 0:
            reasons.append(reason)
    return reasons
  def sia_return_all_cuts(
    subsystem: Subsystem,
    repertoire_distance: Optional[str] = None,
    directions: Optional[Iterable[Direction]] = None,
    partition_scheme: Optional[str] = None,
    partitions: Optional[Iterable] = None,
    system_state: Optional[SystemStateSpecification] = None,
    **kwargs,
    ) -> SystemIrreducibilityAnalysis:
    """Find the minimum information partition of a system."""
    partition_scheme = fallback(partition_scheme, config.SYSTEM_PARTITION_TYPE)

    # TODO(4.0): trivial reducibility

    if partitions is None:
        filter_func = None
        if partitions == "GENERAL":

            def is_disconnecting_partition(partition):
                # Special case for length 1 subsystems so complete partition is included
                return (
                    not connectivity.is_strong(subsystem.apply_cut(partition).proper_cm)
                ) or len(subsystem) == 1

            filter_func = is_disconnecting_partition

        partitions = system_partitions(
            subsystem.node_indices,
            node_labels=subsystem.node_labels,
            partition_scheme=partition_scheme,
            filter_func=filter_func,
        )

    if system_state is None:
        system_state = system_intrinsic_information(
            subsystem, directions=directions
        )

    def _null_sia(**kwargs):
        return NullSystemIrreducibilityAnalysis(
            system_state=system_state,
            node_indices=subsystem.node_indices,
            node_labels=subsystem.node_labels,
            **kwargs,
        )

    if config.SHORTCIRCUIT_SIA:
        shortcircuit_reasons = _has_no_cause_or_effect(system_state)
        if shortcircuit_reasons:
            return _null_sia(reasons=shortcircuit_reasons)

    default_sia = _null_sia(reasons=[ShortCircuitConditions.NO_VALID_PARTITIONS])

    parallel_kwargs = conf.parallel_kwargs(config.PARALLEL_CUT_EVALUATION, **kwargs)
    sias = MapReduce(
        evaluate_partition,
        partitions,
        map_kwargs=dict(
            subsystem=subsystem,
            system_state=system_state,
            repertoire_distance=repertoire_distance,
            directions=directions,
        ),
        shortcircuit_func=utils.is_falsy,
        desc="Evaluating partitions",
        **parallel_kwargs,
    ).run()

    # Find MIP in one pass, keeping track of ties
    # TODO(ties) refactor into resolve_ties module
    mip_sia = default_sia
    mip_key = (float("inf"), float("-inf"))
    ties = [default_sia]
    for sia__ in sias:
        distribution.append(sia__.phi)
    if phi_dist :
      return distribution
    for candidate_mip_sia in sias:
        candidate_key = sia_minimization_key(candidate_mip_sia)
        if candidate_key < mip_key:
            mip_sia = candidate_mip_sia
            mip_key = candidate_key
            ties = [mip_sia]
        elif candidate_key == mip_key:
            ties.append(candidate_mip_sia)
    for tied_mip in ties:
        tied_mip.set_ties(ties)
    return mip_sia
  pyphi.new_big_phi.sia = sia_return_all_cuts
  subsystem.sia()
  return distribution


#### NORMALIZED PHI DISTRIBUTION FUNCTION #####

def get_normalized_phi_distribution(network, state):
  subsystem = pyphi.Subsystem(network,state)
  distribution = []

  def _has_no_cause_or_effect(system_state):
    reasons = []
    for direction, reason in zip(
        Direction.both(),
        [ShortCircuitConditions.NO_CAUSE, ShortCircuitConditions.NO_EFFECT],
    ):
        if system_state[direction].intrinsic_information <= 0:
            reasons.append(reason)
    return reasons

  def sia_return_all_cuts(
    subsystem: Subsystem,
    repertoire_distance: Optional[str] = None,
    directions: Optional[Iterable[Direction]] = None,
    partition_scheme: Optional[str] = None,
    partitions: Optional[Iterable] = None,
    system_state: Optional[SystemStateSpecification] = None,
    **kwargs,
    ) -> SystemIrreducibilityAnalysis:
    """Find the minimum information partition of a system."""
    partition_scheme = fallback(partition_scheme, config.SYSTEM_PARTITION_TYPE)

    # TODO(4.0): trivial reducibility

    if partitions is None:
        filter_func = None
        if partitions == "GENERAL":

            def is_disconnecting_partition(partition):
                # Special case for length 1 subsystems so complete partition is included
                return (
                    not connectivity.is_strong(subsystem.apply_cut(partition).proper_cm)
                ) or len(subsystem) == 1

            filter_func = is_disconnecting_partition

        partitions = system_partitions(
            subsystem.node_indices,
            node_labels=subsystem.node_labels,
            partition_scheme=partition_scheme,
            filter_func=filter_func,
        )

    if system_state is None:
        system_state = system_intrinsic_information(
            subsystem, directions=directions
        )

    def _null_sia(**kwargs):
        return NullSystemIrreducibilityAnalysis(
            system_state=system_state,
            node_indices=subsystem.node_indices,
            node_labels=subsystem.node_labels,
            **kwargs,
        )

    if config.SHORTCIRCUIT_SIA:
        shortcircuit_reasons = _has_no_cause_or_effect(system_state)
        if shortcircuit_reasons:
            return _null_sia(reasons=shortcircuit_reasons)

    default_sia = _null_sia(reasons=[ShortCircuitConditions.NO_VALID_PARTITIONS])

    parallel_kwargs = conf.parallel_kwargs(config.PARALLEL_CUT_EVALUATION, **kwargs)
    sias = MapReduce(
        evaluate_partition,
        partitions,
        map_kwargs=dict(
            subsystem=subsystem,
            system_state=system_state,
            repertoire_distance=repertoire_distance,
            directions=directions,
        ),
        shortcircuit_func=utils.is_falsy,
        desc="Evaluating partitions",
        **parallel_kwargs,
    ).run()

    # Find MIP in one pass, keeping track of ties
    # TODO(ties) refactor into resolve_ties module
    mip_sia = default_sia
    mip_key = (float("inf"), float("-inf"))
    ties = [default_sia]
    for sia__ in sias:
        distribution.append(sia__.normalized_phi)
    if phi_dist :
      return distribution
    for candidate_mip_sia in sias:
        candidate_key = sia_minimization_key(candidate_mip_sia)
        if candidate_key < mip_key:
            mip_sia = candidate_mip_sia
            mip_key = candidate_key
            ties = [mip_sia]
        elif candidate_key == mip_key:
            ties.append(candidate_mip_sia)
    for tied_mip in ties:
        tied_mip.set_ties(ties)
    return mip_sia
  pyphi.new_big_phi.sia = sia_return_all_cuts
  subsystem.sia()
  return distribution

def generate_restricted_partitions(node_indices, node_labels=None):
    """Generate all unidirectional set partitions of a set of nodes."""


    ### Trivial Cut
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)


    directions = [True, False]
    _node_indices = set(range(len(node_indices)))
    for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
        for direction_seq in product(directions, repeat=len(partition)):


            ### Fill out cut matrix
            cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
            for part, direction in zip(partition, direction_seq):
                nonpart = list(_node_indices - set(part))
                if direction:
                    source, target = nonpart, part
                else:
                    source, target = part, nonpart
                cut_matrix[np.ix_(source, target)] = 1


                ### Yield the cuts
            yield GeneralSetPartition(
                node_indices,
                cut_matrix,
                node_labels=node_labels,
                set_partition=partition,
            )


### Yield the unique partitions only
@system_partition_types.register("UNIDIRECTIONAL_PARTITIONS")
@functools.wraps(generate_restricted_partitions)
def unidirectional_set_partitions(node_indices, node_labels=None):
    # TODO(4.0) generate properly without using set
    yield from unique(
        generate_restricted_partitions(node_indices, node_labels=node_labels)
    )



### Generate All Possible Bi-Partitions Of A Set Of Nodes
def get_bipartitions(collection):
    collection = list(collection)
    n = len(collection)
    # Special case: no bipartitions possible if the collection has fewer than 2 elements
    if n < 2:
        return
    # Generate all combinations for the first subset of size from 1 to n-1
    for i in range(1, n):
        for first_subset in combinations(collection, i):
            second_subset = [item for item in collection if item not in first_subset]
            yield [list(first_subset), second_subset]






def create_noisy_network(size):
    ### Obtains a random network of nodes of a given size, directly ready for computing Phi.
    def normalize_rows(matrix):
        ### Scales probabilities in a matrix so that they satisfy conditional independence
        num_rows = matrix.shape[0]
        normalized_matrix = np.zeros_like(matrix)# Create an empty matrix of the same shape
        for i in range(num_rows):
            row = matrix[i, :]
            current_sum = np.sum(row)
            if current_sum > 0:
                scaling_factor = 1.0 / current_sum
                normalized_row = row * scaling_factor
                normalized_matrix[i, :] = normalized_row
            else: print("Zero_Sum_Error: Problem With Values Generated")
        return normalized_matrix


    return pyphi.Network(normalize_rows(pyphi.convert.state_by_node2state_by_state(np.random.rand(2**size,size))))



### Want to generate the plots of normphi, for each partition generation scheme
size = 5
state = [np.random.choice([0,1]) for i in range(size)]
network = create_noisy_network(size)
subsystem = pyphi.Subsystem(network,state)
phi_dist = False
print(subsystem.sia())
phi_dist =  True

### k-partitions
pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI"
normalized_phi_vals = get_normalized_phi_distribution(network, state)
plt.hist(normalized_phi_vals, bins = 1000)


In [ ]:


@system_partition_types.register("K-PARTITIONS")
def _unidirectional_set_partitions2(node_indices, node_labels=None):
    """Generate all unidirectional set partitions of a set of nodes."""


    ### The Trivial Cut: No Partitions Made
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)



    _node_indices = set(range(len(node_indices)))

    for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
        for directions in product(Direction.all(), repeat=len(partition)):
            if Direction.CAUSE in directions and Direction.EFFECT in directions:

                ### Fill out cut Matrix for a partition-direction grouping
              cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
              for part, direction in zip(partition, directions):
                  nonpart = list(_node_indices - set(part))
                  if direction == Direction.CAUSE:
                      source, target = nonpart, part
                  else:
                      source, target = part, nonpart
                  cut_matrix[np.ix_(source, target)] = 1
                  if direction == Direction.BIDIRECTIONAL:
                      cut_matrix[np.ix_(target, source)] = 1


               ### Yield all the cuts
              yield GeneralSetPartition(
                  node_indices,
                  cut_matrix,
                  node_labels=node_labels,
                  set_partition=partition)

              ### Yield the cut ignored by the if (both directions in set)
    yield GeneralSetPartition(
                node_indices,
                np.ones(     (len(_node_indices), len(_node_indices))) - np.identity(len(_node_indices)),
                node_labels=node_labels,
                set_partition=partition,
            )



### Testing Difference in speed:
size = 5
state = [np.random.choice([0,1]) for i in range(size)]
network = create_noisy_network(size)
subsystem = pyphi.Subsystem(network,state)

pyphi.config.SYSTEM_PARTITION_TYPE = "K-PARTITIONS"
s = time.time()
print("My Version: ",subsystem.sia())
f = time.time()
print("Time Taken: ",f-s)


pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI"
s = time.time()
print("Previous Version", subsystem.sia())
f = time.time()
print("Time Taken: ",f-s)


My Version:  [0.04941651618704405, 0.03145276273205432, 0.06738026964203378, 0.03954223871002862, 0.06203705372218766, 0.017047423697869588, 0.019188476221747432, 0.03855268223941981, -0.0001757297959249352, 0.030747876830653213, 0.07067447186225162, 0.03611608127352165, 0.01962334259161625, 0.033022748740766425, 0.047365459605553, 0.03188531278570982, 0.02407315628723104, 0.03347466832027166, 0.02951380185059503, 0.02944452881639612, 0.024983105051276322, 0.044591706062162555, 0.024890744382189856, 0.0351268784404228, 0.03368050674911785, 0.04501454013532122, 0.025759273738041998, 0.048431736041689544, 0.036071332746991126, 0.037999523249317005, 0.06675614471828678, 0.03509886468185148, 0.033941950380455964, 0.05614360754352949, 0.04250854487453678, 0.04614430754137859, 0.03703542799815406, 0.04576167240350582, 0.034236816270645044, 0.044118149108064796, 0.041515002561108405, 0.04547649496220742, 0.03561567487713265, 0.030359020797438664, 0.07639338951597419, 0.013746629530156354, 0.1